<a id='1.3'></a>
## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor
from sklearn.preprocessing import StandardScaler #平均值是0, 標準差是1
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
import sys
sys.path.append("../FinRL")
import itertools

D:\anaconda3\envs\RL-GAT\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## Create Folders

In [2]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    CHIPS,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [3]:
TRAIN_START_DATE = '2013-01-01'
TRAIN_END_DATE = '2021-11-01'
TRADE_START_DATE = '2021-11-01'
TRADE_END_DATE = '2024-05-20'
PREDICT_START_DATE = '2024-05-01'
PREDICT_END_DATE = '2024-06-25'

In [4]:
df = pd.read_csv('Input for Stock-Picked Agent train.csv')
df = df.fillna(0)

In [5]:
features = ['CostOfGoodsSold',
            'EPS',
            'EquityAttributableToOwnersOfParent_x',
            'IncomeAfterTaxes',
            'IncomeFromContinuingOperations',
            'OtherComprehensiveIncome',
            'Revenue',
            'TAX',
            'TotalConsolidatedProfitForThePeriod',	
            'CapitalStock',
            'CapitalStock_per',	
            'CapitalSurplus',
            'CapitalSurplus_per',	
            'CashAndCashEquivalents',	
            'CashAndCashEquivalents_per',	
            'CurrentAssets',
            'CurrentAssets_per',	
            'Equity',
            'EquityAttributableToOwnersOfParent_y',
            'EquityAttributableToOwnersOfParent_per',	
            'Equity_per',
            'NoncurrentAssets',
            'NoncurrentAssets_per',	
            'NoncurrentLiabilities',
            'NoncurrentLiabilities_per',	
            'OrdinaryShare',
            'OrdinaryShare_per',	
            'OtherCurrentLiabilities',	
            'OtherCurrentLiabilities_per',	
            'OtherEquityInterest',
            'OtherEquityInterest_per',	
            'RetainedEarnings',
            'RetainedEarnings_per',	
            'TotalAssets',
            'TotalAssets_per',	
            'CashBalancesBeginningOfPeriod',
            'CashBalancesEndOfPeriod',
            'Depreciation',
            'PayTheInterest',	
            'PropertyAndPlantAndEquipment' ]

# 建立 StandardScaler 物件
ss = StandardScaler()

df_features_scaled = pd.DataFrame(ss.fit_transform(df[features]), columns=features, index=df.index)


df.update(df_features_scaled)

In [6]:
fe = FeatureEngineer(
                    use_technical_indicator=False,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False, 
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

In [7]:
processed

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,64,65,66,67,68,69,70,71,72,73
0,2013-08-15,1101.TW,-0.247169,-0.310909,-0.139828,-0.082760,-0.082955,-0.231882,-0.259529,-0.107094,...,0.034699,0.000000,0.034699,0.034699,0.034699,0.000000,0.000000,0.000000,0.034699,0.000000
0,2013-08-15,1102.TW,-0.295657,-0.287713,-0.144827,-0.133388,-0.133584,-0.106698,-0.323279,-0.305028,...,0.000000,0.000000,0.000000,0.000000,0.034107,0.000000,0.000000,0.000000,0.000000,0.000000
0,2013-08-15,1216.TW,0.081170,-0.330536,-0.123086,-0.005903,-0.006096,0.072600,0.213781,0.207958,...,0.000000,0.000000,0.000000,0.034107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,2013-08-15,1229.TW,-0.390763,-0.366223,-0.270142,-0.285492,-0.285693,-0.253592,-0.425707,-0.376792,...,0.000000,0.033570,0.034107,0.000000,0.000000,0.000000,0.000000,0.034107,0.034107,0.000000
0,2013-08-15,1301.TW,-0.056654,-0.355517,-0.110017,-0.123587,-0.123783,3.549250,-0.091662,-0.013343,...,0.000000,0.033255,0.034327,0.034327,0.034327,0.033495,0.034327,0.000000,0.000000,0.034327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,2024-05-16,9904.TW,-0.088720,-0.153889,-0.020438,0.075105,0.074915,1.514009,-0.049122,0.104701,...,0.000000,0.000000,0.000000,0.033770,0.000000,0.000000,0.033801,0.000000,0.000000,0.033806
43,2024-05-16,9910.TW,-0.291552,-0.205634,-0.159577,-0.226485,-0.226685,0.179905,-0.308479,-0.223199,...,0.000000,0.033770,0.033770,0.033770,0.033807,0.033770,0.000000,0.033801,0.033790,0.000000
43,2024-05-16,9921.TW,-0.314350,-0.198497,-0.230556,-0.270764,-0.270965,0.090426,-0.336153,-0.311561,...,0.000000,0.033770,0.000000,0.000000,0.000000,0.033770,0.033801,0.000000,0.000000,0.000000
43,2024-05-16,9941.TW,-0.365917,-0.030771,-0.213942,-0.227109,-0.227309,0.075986,-0.370593,-0.247142,...,0.000000,0.000000,0.000000,0.000000,0.033807,0.033770,0.033801,0.000000,0.033790,0.033806


## Data Split

In [8]:
train = data_split(processed, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed, TRADE_START_DATE,TRADE_END_DATE)
PREDICT = data_split(processed, PREDICT_START_DATE,PREDICT_END_DATE)

print(len(train))
print(len(trade))
print(len(PREDICT))

2442
814
74


In [9]:
stock_dimension = len(train.tic.unique()) 
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 74, State Space: 8585


In [10]:
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
num_stock_shares = [0] * stock_dimension


env_kwargs = {
    "hmax": 1,
    "initial_amount": 0,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 0
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)


In [11]:
env_train, _ = e_train_gym.get_sb_env()

D:\anaconda3\envs\RL-GAT\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [12]:
agent = DRLAgent(env = env_train)

### Agent: PPO

In [13]:
df_account_value_ppo = pd.DataFrame()

agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 2048,
}
#model_ppo = agent.get_model("ppo")
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)
# set up logger
tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)

trained_ppo = agent.train_model(model=model_ppo, 
                         tb_log_name='ppo',
                         total_timesteps=3440)  

Get Model kwargs:  {'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 2048}
Using cpu device
{'pi': [512, 512], 'vf': [512, 512]}
Logging to results/ppo
------------------------------------
| time/              |             |
|    fps             | 11          |
|    iterations      | 1           |
|    time_elapsed    | 175         |
|    total_timesteps | 2048        |
| train/             |             |
|    reward          | 0.038914252 |
------------------------------------
day: 32, episode: 100
begin_total_asset: 0.00
end_total_asset: 4.66
total_reward: 4.66
total_cost: 0.00
total_trades: 1133
Sharpe: nan
------------------------------------------
| time/                   |              |
|    fps                  | 12           |
|    iterations           | 2            |
|    time_elapsed         | 322          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.030148903  |
|    clip_

In [ ]:
trained_ppo.save("PPO_Stock_Picked_Agent_Sharpe_Ratio.zip")

In [14]:
trade

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,64,65,66,67,68,69,70,71,72,73
0,2021-11-15,1101.TW,-0.255993,-0.330536,-0.059052,-0.057384,-0.088853,0.369580,-0.270401,-0.040512,...,0.033743,0.033743,0.033753,0.033778,0.000000,0.00000,0.000000,0.000000,0.033778,0.033778
0,2021-11-15,1102.TW,-0.284413,-0.250242,-0.095434,-0.083920,-0.084115,-0.086845,-0.300987,-0.018814,...,0.000000,0.000000,0.033768,0.033773,0.033773,0.00000,0.033848,0.000000,0.033773,0.000000
0,2021-11-15,1216.TW,0.153784,-0.257379,0.040982,0.173654,0.173467,-0.147669,0.325466,0.351185,...,0.000000,0.033740,0.033753,0.000000,0.033778,0.00000,0.000000,0.033739,0.000000,0.033778
0,2021-11-15,1229.TW,-0.384333,-0.359085,-0.183358,-0.252774,-0.252974,0.221721,-0.418038,-0.388316,...,0.033772,0.000000,0.033776,0.033781,0.000000,0.00000,0.033808,0.000000,0.033781,0.033781
0,2021-11-15,1301.TW,-0.069801,0.104837,0.826938,0.822674,0.822509,1.189769,-0.026953,0.757792,...,0.000000,0.000000,0.033774,0.033779,0.000000,0.00000,0.000000,0.000000,0.000000,0.033779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2024-05-16,9904.TW,-0.088720,-0.153889,-0.020438,0.075105,0.074915,1.514009,-0.049122,0.104701,...,0.000000,0.000000,0.000000,0.033770,0.000000,0.00000,0.033801,0.000000,0.000000,0.033806
10,2024-05-16,9910.TW,-0.291552,-0.205634,-0.159577,-0.226485,-0.226685,0.179905,-0.308479,-0.223199,...,0.000000,0.033770,0.033770,0.033770,0.033807,0.03377,0.000000,0.033801,0.033790,0.000000
10,2024-05-16,9921.TW,-0.314350,-0.198497,-0.230556,-0.270764,-0.270965,0.090426,-0.336153,-0.311561,...,0.000000,0.033770,0.000000,0.000000,0.000000,0.03377,0.033801,0.000000,0.000000,0.000000
10,2024-05-16,9941.TW,-0.365917,-0.030771,-0.213942,-0.227109,-0.227309,0.075986,-0.370593,-0.247142,...,0.000000,0.000000,0.000000,0.000000,0.033807,0.03377,0.033801,0.000000,0.033790,0.033806


In [16]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col=None, **env_kwargs)

trained_moedl =trained_ppo
df_account_value, df_actions ,df_balance= DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

print("==============Get Backtest Results===========")

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all.csv")


D:\anaconda3\envs\RL-GAT\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


hit end!
==============Get Backtest Results===========
Annual return          inf
Cumulative returns     inf
Annual volatility      NaN
Sharpe ratio           NaN
Calmar ratio           NaN
Stability              NaN
Max drawdown          -inf
Omega ratio            0.0
Sortino ratio          NaN
Skew                   NaN
Kurtosis               NaN
Tail ratio             NaN
Daily value at risk    NaN
dtype: float64


D:\anaconda3\envs\RL-GAT\lib\site-packages\numpy\lib\nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
D:\anaconda3\envs\RL-GAT\lib\site-packages\numpy\lib\nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
D:\anaconda3\envs\RL-GAT\lib\site-packages\empyrical\stats.py:396: RuntimeWarning: invalid value encountered in subtract
  nanmin((cumulative - max_return) / max_return, axis=0, out=out)
D:\anaconda3\envs\RL-GAT\lib\site-packages\empyrical\stats.py:396: RuntimeWarning: invalid value encountered in divide
  nanmin((cumulative - max_return) / max_return, axis=0, out=out)
D:\anaconda3\envs\RL-GAT\lib\site-packages\empyrical\stats.py:585: RuntimeWarning: invalid value encountered in scalar divide
  temp = annual_return(
D:\anaconda3\envs\RL-GAT\lib\site-packages\empyrical\stats.py:1494: RuntimeWarning: invalid 

In [17]:
PREDICT

,date,tic,CostOfGoodsSold,EPS,EquityAttributableToOwnersOfParent_x,IncomeAfterTaxes,IncomeFromContinuingOperations,OtherComprehensiveIncome,Revenue,TAX,...,64,65,66,67,68,69,70,71,72,73
0,2024-05-16,1101.TW,-0.256312,-0.389419,-0.176301,-0.173972,-0.174170,1.153565,-0.278526,0.003218,...,0.000000,0.03377,0.00000,0.03377,0.000000,0.03377,0.000000,0.000000,0.03379,0.000000
0,2024-05-16,1102.TW,-0.301756,-0.277007,0.231757,-0.140335,-0.140532,1.253686,-0.333499,-0.103861,...,0.033809,0.03377,0.00000,0.00000,0.033807,0.00000,0.033801,0.000000,0.00000,0.000000
0,2024-05-16,1216.TW,0.304845,-0.262732,0.030492,0.181107,0.180921,0.763385,0.531113,0.518849,...,0.033809,0.00000,0.00000,0.00000,0.033807,0.03377,0.033801,0.000000,0.03379,0.000000
0,2024-05-16,1229.TW,-0.382217,-0.310909,-0.028255,-0.234362,-0.234562,0.963509,-0.415242,-0.387690,...,0.033809,0.00000,0.00000,0.03377,0.000000,0.00000,0.033801,0.000000,0.03379,0.033806
0,2024-05-16,1301.TW,-0.094513,-0.430458,-0.276560,-0.291783,-0.291985,-0.704097,-0.141386,-0.348283,...,0.000000,0.03377,0.00000,0.03377,0.033807,0.00000,0.033801,0.000000,0.00000,0.033806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2024-05-16,9904.TW,-0.088720,-0.153889,-0.020438,0.075105,0.074915,1.514009,-0.049122,0.104701,...,0.000000,0.00000,0.00000,0.03377,0.000000,0.00000,0.033801,0.000000,0.00000,0.033806
0,2024-05-16,9910.TW,-0.291552,-0.205634,-0.159577,-0.226485,-0.226685,0.179905,-0.308479,-0.223199,...,0.000000,0.03377,0.03377,0.03377,0.033807,0.03377,0.000000,0.033801,0.03379,0.000000
0,2024-05-16,9921.TW,-0.314350,-0.198497,-0.230556,-0.270764,-0.270965,0.090426,-0.336153,-0.311561,...,0.000000,0.03377,0.00000,0.00000,0.000000,0.03377,0.033801,0.000000,0.00000,0.000000
0,2024-05-16,9941.TW,-0.365917,-0.030771,-0.213942,-0.227109,-0.227309,0.075986,-0.370593,-0.247142,...,0.000000,0.00000,0.00000,0.00000,0.033807,0.03377,0.033801,0.000000,0.03379,0.033806


In [18]:
e_predict_gym = StockTradingEnv(df = PREDICT, turbulence_threshold = None,risk_indicator_col=None, **env_kwargs)

predict_env, predict_obs = e_predict_gym.get_sb_env()
predict_env.reset()
action, _states = trained_ppo.predict(predict_obs, deterministic=True)

print(action)

[[0 0 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0
  1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 1 1
  0 1]]


D:\anaconda3\envs\RL-GAT\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
